In [1]:
import torch
import pyro
import pyro.distributions as dist
import pyro.distributions.transforms as T
from pyro.nn import AutoRegressiveNN
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
smoke_test = ('CI' in os.environ)

In [2]:
num_bijectors = 6
steps = 5000
lr = 1e-4
datafile = 'machine-1-1.csv'

In [3]:
X_data = pd.read_csv(datafile, header=None)
X_data = X_data.iloc[:10000]
input_dims = X_data.shape[1]

X_data = X_data.values.astype(np.float)
X_data = MinMaxScaler().fit_transform(X_data)

In [4]:
X_train, X_test = train_test_split(X_data, test_size=0.33, random_state=42, shuffle=False)

In [5]:
def local_ploter(data, cols=4):
    fig, ax = plt.subplots(nrows=math.ceil(data.shape[1]/cols), ncols=cols, figsize=(30,10))
    for i in range(math.ceil(data.shape[1]/cols)):
        for j in range(cols):
            if i*cols+j < data.shape[1]:
                ax[i][j].plot(data[:, i*cols+j])
                ax[i][j].set_ylim(0, 1)

    plt.tight_layout()
    plt.show()

In [6]:
base_dist = dist.Normal(torch.zeros(input_dims), torch.ones(input_dims))
# transform = T.spline_coupling(input_dims, count_bins=16)
arn = AutoRegressiveNN(input_dims, [16]*10)
transform = T.AffineAutoregressive(arn)
flow_dist = dist.TransformedDistribution(base_dist, [transform])

In [7]:
dataset = torch.tensor(X_train, dtype=torch.float)
optimizer = torch.optim.Adam(transform.parameters(), lr=lr)
for step in range(steps+1):
    optimizer.zero_grad()
    loss = -flow_dist.log_prob(dataset).mean()
    loss.backward()
    optimizer.step()
    flow_dist.clear_cache()

    if step % 500 == 0:
        print('step: {}, loss: {}'.format(step, loss.item()))

step: 0, loss: 6.228704452514648
step: 500, loss: 5.433954238891602
step: 1000, loss: 4.180740833282471
step: 1500, loss: 1.3124299049377441
step: 2000, loss: 0.12506146728992462
step: 2500, loss: -1.0093340873718262


In [ ]:
X_flow = flow_dist.sample(torch.Size([1000,])).detach().numpy()

In [ ]:
local_ploter(X_flow)